In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

df_gp = df.groupby(['label_value']).mean()
X = list(df_gp.columns)

Y = df_gp[0:]
Z = df_gp[1:]

df = pd.DataFrame(np.c_[Y,Z], index=X)
df.plot.bar()
plt.labels(['0', '1'])

plt.show()